<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#第十周作业" data-toc-modified-id="第十周作业-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>第十周作业</a></span></li><li><span><a href="#模型代码补全与心得体会" data-toc-modified-id="模型代码补全与心得体会-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>模型代码补全与心得体会</a></span><ul class="toc-item"><li><span><a href="#convert_fcn_dataset.py" data-toc-modified-id="convert_fcn_dataset.py-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>convert_fcn_dataset.py</a></span></li><li><span><a href="#train.py" data-toc-modified-id="train.py-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>train.py</a></span></li></ul></li><li><span><a href="#Tinymind上运行" data-toc-modified-id="Tinymind上运行-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Tinymind上运行</a></span><ul class="toc-item"><li><span><a href="#数据集准备" data-toc-modified-id="数据集准备-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>数据集准备</a></span></li><li><span><a href="#模型训练" data-toc-modified-id="模型训练-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>模型训练</a></span></li><li><span><a href="#训练结果" data-toc-modified-id="训练结果-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>训练结果</a></span></li><li><span><a href="#验证图片" data-toc-modified-id="验证图片-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>验证图片</a></span></li></ul></li></ul></div>

# 第十周作业

# 模型代码补全与心得体会

## convert_fcn_dataset.py

In [ ]:
# 定义tf.train.Feature转换, 其中的value为二进制
def int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
def bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def dict_to_tf_example(data, label):
    with open(data, 'rb') as inf:
        encoded_data = inf.read()
    img_label = cv2.imread(label)
    img_mask = image2label(img_label)
    encoded_label = img_mask.astype(np.uint8).tobytes()
    filename = os.path.split(data)[1]

    height, width = img_label.shape[0], img_label.shape[1]
    if height < vgg_16.default_image_size or width < vgg_16.default_image_size:
        # 保证最后随机裁剪的尺寸
        return None

    # Your code here, fill the dict
    feature_dict = {
        'image/height': int64_feature(height),
        'image/width': int64_feature(width),
        'image/filename':bytes_feature(filename.encode('utf8')),
        'image/encoded': bytes_feature(encoded_data),
        'image/label': bytes_feature(encoded_label),
        'image/format': bytes_feature('jpeg'.encode('utf8')),
    }
    example = tf.train.Example(features=tf.train.Features(feature=feature_dict))
    return example

def create_tf_record(output_filename, file_pars):
    # Your code here
    writer = tf.python_io.TFRecordWriter(output_filename)  #生成tfrecord
    for jpg_path,png_path in file_pars:  #对图片和标注文件名列表进行遍历
        try:
            tf_example = dict_to_tf_example(jpg_path, png_path)  #实例化tf_example
            writer.write(tf_example.SerializeToString())  #写入到tfrecord
        except AttributeError:  #若随机裁剪的尺寸过小,tf_example=None,writer写入时会报错
        #AttributeError: 'NoneType' object has no attribute 'SerializeToString'
            logging.warning('random size too small,ignoring *_*')
    writer.close()  #关闭tfrecord


## train.py

In [ ]:
upsample_factor = 8  #修改上采样为 8s



#-------------------------------分割线-----------------------------------------------
#取出pool4
pool4_feature = end_points['vgg_16/pool4']  #32*32*512

with tf.variable_scope('vgg_16/fc8_x2'):  #分类器 32*32*512=>32*32*21
    aux_logits_16s = slim.conv2d(pool4_feature, number_of_classes, [1, 1],
                                 activation_fn=None,
                                 weights_initializer=tf.zeros_initializer,
                                 scope='conv_pool4')

# Perform the upsampling 
upsample_filter_np_x2 = bilinear_upsample_weights(2, number_of_classes)
upsample_filter_tensor_x2 = tf.Variable(upsample_filter_np_x2, name='vgg_16/fc8/t_conv_x2')
# 对['vgg_16/fc8']进行2倍反卷积,与['vgg_16/pool4']等大
#     16*16*21        =>            32*32*21
upsampled_logits_x2 = tf.nn.conv2d_transpose(logits, upsample_filter_tensor_x2,
                                          output_shape=tf.shape(aux_logits_16s),
                                          strides=[1, 2, 2, 1],
                                          padding='SAME')

#相加和  32*32*21 + 32*32*21 = 32*32*21
upsampled_logits_x2 = upsampled_logits_x2 + aux_logits_16s




#取出pool3
pool3_feature = end_points['vgg_16/pool3']  #64*64*256

with tf.variable_scope('vgg_16/fc8_x2x2'):  #分类器 64*64*256=>64*64*21
    aux_logits_8s = slim.conv2d(pool3_feature, number_of_classes, [1, 1],
                                 activation_fn=None,
                                 weights_initializer=tf.zeros_initializer,
                                 scope='conv_pool3')

# Perform the upsampling 
upsample_filter_np_x2x2 = bilinear_upsample_weights(2, number_of_classes)
upsample_filter_tensor_x2x2 = tf.Variable(upsample_filter_np_x2x2, name='vgg_16/fc8/t_conv_x2x2')
# 对'加和结果'再进行2倍反卷积,与['vgg_16/pool3']等大
#   32*32*21        =>            64*64*21
upsampled_logits_x2x2 = tf.nn.conv2d_transpose(upsampled_logits_x2, upsample_filter_tensor_x2x2,
                                          output_shape=tf.shape(aux_logits_8s),
                                          strides=[1, 2, 2, 1],
                                          padding='SAME')

#再加和  64*64*21 + 64*64*21 = 64*64*21
upsampled_logits = upsampled_logits_x2x2 + aux_logits_8s


# Perform the upsampling 
upsample_filter_np_x8 = bilinear_upsample_weights(upsample_factor, number_of_classes)
upsample_filter_tensor_x8 = tf.Variable(upsample_filter_np_x8, name='vgg_16/fc8/t_conv_x8')
# 对加和后的结果8倍反卷积 64*64*21=>512*512*21
# 并进行适当裁剪 512*512*21=>500*500*21
upsampled_logits = tf.nn.conv2d_transpose(upsampled_logits, upsample_filter_tensor_x8,
                                          output_shape=upsampled_logits_shape,
                                          strides=[1, upsample_factor, upsample_factor, 1],
                                          padding='SAME')
#--------------------------------------------------------------------------------------

# Tinymind上运行

FCN_8s(本次作业): https://www.tinymind.com/executions/4gisp4be

## 数据集准备
![dataet_w9.png](https://github.com/zhouzhiqi/Deep-Learning/tree/master/HomeWork/tmp/dataet_w9.png)

## 模型训练
部分运行结果截取:
>2018-05-06 08:16:04,106 - DEBUG - fcn_8s.py:325 - step 1290 Current Loss: 88.31440734863281   
2018-05-06 08:16:04,106 - DEBUG - fcn_8s.py:327 - [9.80] imgs/s   
2018-05-06 08:16:20,299 - DEBUG - fcn_8s.py:325 - step 1300 Current Loss: 80.02828216552734   
2018-05-06 08:16:20,300 - DEBUG - fcn_8s.py:327 - [9.88] imgs/s   
2018-05-06 08:16:22,189 - DEBUG - fcn_8s.py:334 - Model saved in file: /output/train/model.ckpt-1300   
2018-05-06 08:16:38,218 - DEBUG - fcn_8s.py:325 - step 1310 Current Loss: 72.52936553955078    
2018-05-06 08:16:38,219 - DEBUG - fcn_8s.py:327 - [8.93] imgs/s   
2018-05-06 08:16:54,636 - DEBUG - fcn_8s.py:325 - step 1320 Current Loss: 102.80619812011719    
2018-05-06 08:16:54,636 - DEBUG - fcn_8s.py:327 - [9.75] imgs/s   

## 训练结果
Tinymind上运行结果地址: https://www.tinymind.com/executions/4gisp4be/output

## 验证图片
原图:
![val_1400_img.jpg](https://github.com/zhouzhiqi/Deep-Learning/tree/master/HomeWork/tmp/val_1400_img.jpg)

标签:
![val_1400_annotation.jpg](https://github.com/zhouzhiqi/Deep-Learning/tree/master/HomeWork/tmp/val_1400_annotation.jpg)

预测:
![val_1400_prediction.jpg](https://github.com/zhouzhiqi/Deep-Learning/tree/master/HomeWork/tmp/val_1400_prediction.jpg)

着色:
![val_1400_overlay.jpg](https://github.com/zhouzhiqi/Deep-Learning/tree/master/HomeWork/tmp/val_1400_overlay.jpg)

CRF:
![val_1400_prediction_crfed.jpg](https://github.com/zhouzhiqi/Deep-Learning/tree/master/HomeWork/tmp/val_1400_prediction_crfed.jpg)